# Symbiotic Relationships Simulation

In [ ]:
from mesa.discrete_space import CellAgent, FixedAgent
import math
import mesa
from mesa import Model
from mesa.datacollection import DataCollector
from mesa.discrete_space import OrthogonalVonNeumannGrid, CellAgent, FixedAgent
from mesa.experimental.devs import ABMSimulator


class Frog():
    def __init__(self):
        pass


The frog makes sound
Ribbit..
